The Primo Insurance Company is introducing two new product lines: special risk insurance and mortgages. The expected profit is \$5 per unit on special risk insurance and \$2 per unit on mortgages. Management wishes to establish sales quotas for the new product lines to maximize total expected profit. The work requirements are as follows:

| Department |  Special Risk | Mortgage | Work-Hours Available |
| :--- | :---: | :---: | :---: |
| Underwriting | 3 | 2 | 2400 |
| Administration | 0 | 1 | 800 |
| Claims | 2 | 0 | 1200 |

In [1]:
from pyomo.environ import *

In [2]:
model = AbstractModel()

## Sets

In [3]:
model.products = Set()
model.departments = Set()

## Parameters

In [4]:
model.products_profit = Param(model.products)
model.work_hours_available = Param(model.departments)
model.work_hours_per_unit = Param(model.products, model.departments)

## Variables

In [5]:
model.quotas = Var(model.products, within=NonNegativeReals)

## Objective

In [6]:
def total_profit(model):
    return summation(model.products_profit, model.quotas)


model.final_profit = Objective(rule=total_profit, sense=maximize)

## Constraints


In [7]:
def max_work_hours_per_department(model, department):
    return sum(model.work_hours_per_unit[p,department] * model.quotas[p] for p in model.products) <= model.work_hours_available[department]


model.departments_constraint = Constraint(model.departments, rule=max_work_hours_per_department)

## Instantiate the Model

In [8]:
data = { None : { 'products' : { None : ('Special Risk', 'Mortgage') }, 
                  'departments' : { None : ('Underwriting', 'Administration', 'Claims') },
                  'products_profit' : { 'Special Risk' : 5, 'Mortgage' : 2 },
                  'work_hours_available' : { 'Underwriting' : 2400, 'Administration' : 800, 'Claims' : 1200 },
                  'work_hours_per_unit' : { ('Special Risk', 'Underwriting') : 3, 
                                            ('Special Risk', 'Administration') : 0,
                                            ('Special Risk', 'Claims') : 2, 
                                            ('Mortgage', 'Underwriting') : 2, 
                                            ('Mortgage', 'Administration') : 1,
                                            ('Mortgage', 'Claims') : 0 }

                }
        }

In [9]:
instance = model.create_instance(data)
instance.pprint()

3 Set Declarations
    departments : Dim=0, Dimen=1, Size=3, Domain=None, Ordered=False, Bounds=None
        ['Administration', 'Claims', 'Underwriting']
    products : Dim=0, Dimen=1, Size=2, Domain=None, Ordered=False, Bounds=None
        ['Mortgage', 'Special Risk']
    work_hours_per_unit_index : Dim=0, Dimen=2, Size=6, Domain=None, Ordered=False, Bounds=None
        Virtual

3 Param Declarations
    products_profit : Size=2, Index=products, Domain=Any, Default=None, Mutable=False
        Key          : Value
            Mortgage :     2
        Special Risk :     5
    work_hours_available : Size=3, Index=departments, Domain=Any, Default=None, Mutable=False
        Key            : Value
        Administration :   800
                Claims :  1200
          Underwriting :  2400
    work_hours_per_unit : Size=6, Index=work_hours_per_unit_index, Domain=Any, Default=None, Mutable=False
        Key                                : Value
            ('Mortgage', 'Administration') :   

## Solve the problem

In [10]:
opt = SolverFactory('glpk')

In [11]:
results = opt.solve(instance)
instance.solutions.store_to(results)
print(results)


Problem: 
- Name: unknown
  Lower bound: 3600.0
  Upper bound: 3600.0
  Number of objectives: 1
  Number of constraints: 4
  Number of variables: 3
  Number of nonzeros: 5
  Sense: maximize
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.01265406608581543
Solution: 
- number of solutions: 1
  number of solutions displayed: 1
- Gap: 0.0
  Status: feasible
  Message: None
  Objective:
    final_profit:
      Value: 3600
  Variable:
    quotas[Mortgage]:
      Value: 300
    quotas[Special Risk]:
      Value: 600
  Constraint: No values

